# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

In [13]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import copy
import trimesh as tm
import pandas as pd

In [14]:
# create all possible configurations
l_bis = []
for i in range(2**8):
    bi = np.array(list(np.binary_repr(i, width=8))).astype(int).reshape((2,2,2))
    l_bi = tg.to_lattice(bi, [0,0,0])
    l_bis.append(l_bi)

In [40]:
#  creating stencils
so_base = tg.create_stencil("von_neumann", 0, 1)
sx_base = copy.deepcopy(so_base)
sx_base.set_index((0,0,0), 0)
sx_base.set_index((1,0,0), 1)

# setting separate direction
corner_state = dict(
    OO = so_base,
    XP = sx_base,
    XN = np.flip(sx_base, 0),
    YP = np.transpose(sx_base, (1,0,2)),
    YN = np.transpose(np.flip(sx_base, 0), (1,0,2)),
    ZP = np.transpose(sx_base, (2,1,0)),
    ZN = np.transpose(np.flip(sx_base, 0), (2,1,0))
)

for key, value in corner_state.items():
    value.function = tg.sfunc.sum
    corner_state[key] = value

# setting paired directions
corner_state["XX"] = corner_state["XN"] + corner_state["XP"]
corner_state["YY"] = corner_state["YN"] + corner_state["YP"]
corner_state["ZZ"] = corner_state["ZN"] + corner_state["ZP"]

# dummy symmetry string 
# sym_str = [["OO"], ["XP"], ["XN"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
sym_str = [["OO"], ["XX", "YY"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX", "YY"], ["ZZ"]]
# sym_str = [["OO"], ["XX", "YY", "ZZ"]]
# sym_str = [["OO"]]

stencils = []

for i, sym in enumerate(sym_str):
    # init the symmetry stencil with the first item
    sym_stencil = copy.deepcopy(corner_state[sym[0]])
    # iteratively add the rest of the symmetry stencils together
    for key in sym[1:]:
        sym_stencil += corner_state[key]
    # append
    stencils.append(sym_stencil)
    # compute the sum of all stencils
    if i == 0:
        stencil_sum = copy.deepcopy(sym_stencil)
    else:
        stencil_sum += sym_stencil

# check if there is any neighbours that are not included
stencil_rest = tg.create_stencil("von_neumann", 1, 1) - stencil_sum

# print(np.zeros((3,3,3)).sum())
if (np.array(stencil_rest).sum()):
    stencil_rest.function = tg.sfunc.sum
    stencils.append(stencil_rest)

# add the sum function to all stencils
for s in stencils:
    s.function = tg.sfunc.sum

In [16]:
results = []
for l in l_bis:
    # apply the stencils in all corners of all possible configurations
    r_s = [l.apply_stencil(s).flatten() for s in stencils]

    # concatenate results into columns
    result = np.c_[tuple(r_s)]

    results.append(result)

# stack results vertically
res_stacked = np.vstack(results)

# find the uniqe arangements of corners
uniq_corner_arang = np.unique(res_stacked, axis = 0)
print(uniq_corner_arang)
print(len(uniq_corner_arang))

[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 2 0 0]
 [0 2 0 1]
 [0 2 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 2 0 0]
 [1 2 0 1]
 [1 2 1 0]]
18


In [17]:
stencil_ind = []
for s in stencils:
    filled_ind = np.array(np.where(np.array(s)==1)).T
    #relocate to corner
    filled_ind_reloc = filled_ind - 1
    # flip vertically
    filled_ind_flpd = np.flip(filled_ind_reloc, 0).astype(int)

    stencil_ind.append(filled_ind_flpd)
print(stencil_ind)

[array([[0, 0, 0]]), array([[ 1,  0,  0],
       [ 0,  1,  0],
       [ 0, -1,  0],
       [-1,  0,  0]]), array([[0, 0, 1]]), array([[ 0,  0, -1]])]


In [18]:
# generate mini-lattices to represent the geometrical equivalent of unique corner arrangments
base_zero = np.zeros((2,2,2), dtype=np.int8)

corner_neigh_lattices = []
corner_loc_lattices = []
# for each unique arangement, create a representation
for uca in uniq_corner_arang:
    corner_arang = np.copy(base_zero)
    corner_loc = np.copy(base_zero)
    corner_loc[0,0,0] = 1

    shift = np.zeros(3)
    # for each list of indices of each stencil
    for i, s_inds in enumerate(stencil_ind):
        # for each index
        for j, s_i in enumerate(s_inds):
            # check if the sum is big enough to mark this index
            if uca[i] > j:
                # check if the index is positive
                if s_i.sum() >= 0:
                    # mark as one
                    corner_arang[tuple(s_i)] = 1
                # if the index is negative
                else:
                    # mark as one
                    corner_arang[tuple(s_i)] = 1
                    # record the index as a shift
                    shift += s_i

    # roll nwighbourhood and locations
    corner_arang = np.roll(corner_arang, tuple(shift.astype(int)), (0,1,2))
    corner_loc = np.roll(corner_loc, tuple(shift.astype(int)), (0,1,2))

    # convert to lattice
    corner_lat = tg.to_lattice(corner_arang, np.zeros(3))
    corner_neigh_lattices.append(corner_lat)

    corner_loc_lat = tg.to_lattice(corner_loc, np.zeros(3))
    corner_loc_lattices.append(corner_loc_lat)

In [19]:
p = pv.Plotter(notebook=True)

base_lattice = corner_neigh_lattices[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# # adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = corner_neigh_lattices[f]
    loc = corner_loc_lattices[f]

    # Add the data values to the cell data
    grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="filled")
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=0.7, show_scalar_bar=False)
    
    # Add the data values to the cell data
    grid.cell_arrays["corner"] = loc.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="corner")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=1.0, show_scalar_bar=False, color="white")

    return

p.add_slider_widget(create_mesh, [1, len(corner_neigh_lattices)], title='Arrangements', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(4.363703305156274, 4.363703305156274, 4.363703305156274),
 (0.5, 0.5, 0.5),
 (0.0, 0.0, 1.0)]

In [20]:
def to_csv(lattice, filepath: str):
        """This method saves the lattice to a csv file

        Args:
            filepath: path to the csv file
        """
        # volume to panda dataframe
        vol_df = lattice.to_panadas()

        # specifying metadata and transposing it
        metadata = pd.DataFrame({
            'minbound': lattice.minbound,
            'shape': np.array(lattice.shape),
            'unit': lattice.unit,
        })

        with open(filepath, 'w') as df_out:

            metadata.to_csv(df_out, index=False,header=True, float_format='%g', line_terminator='\n')

            df_out.write('\n')

            vol_df.to_csv(df_out, index=False, float_format='%g', line_terminator='\n')

In [21]:
interior_zero = tg.to_lattice(np.zeros((2,2,2), dtype=np.int8), [0,0,0])
border_pad = np.pad(interior_zero, (1,1), 'constant', constant_values=(1))
base_zero = tg.to_lattice(np.zeros((4,4,4), dtype=np.int8), [0,0,0])
s = tg.create_stencil("von_neumann", 1,1)
s.set_index([0,0,0], 0)

for i, core, neigh in zip(range(len(corner_loc_lattices)), corner_loc_lattices, corner_neigh_lattices):
    # construct saving paths
    core_path = os.path.relpath('../data/bmc_templates/core_' + f'{i:02}' + '.csv')
    neigh_path = os.path.relpath('../data/bmc_templates/neighs_' + f'{i:02}' + '.csv')
    e_neigh_path = os.path.relpath('../data/bmc_templates/e_neighs_' + f'{i:02}' + '.csv')

    # aggregate neighbours
    neigh = tg.to_lattice(np.pad(neigh, (1,1), 'constant', constant_values=(0)), [0,0,0])

    # construct the padded core
    core_pad = tg.to_lattice(np.pad(core, (1,1), 'constant', constant_values=(0)), [0,0,0])
    # extract the outer neighbours of the core
    core_3ind = np.array(np.where(core_pad==1)).flatten()
    pals = base_zero.find_neighbours_masked(s, loc=core_3ind)
    extra_neighs = np.copy(base_zero).flatten()
    # set the extra neighbours as the current state of the core
    extra_neighs[pals] = neigh[tuple(core_3ind)]
    extra_neighs = tg.to_lattice(extra_neighs.reshape(4,4,4), [0,0,0])
    # remove interior pals
    extra_neighs *= border_pad


    # save to csv
    to_csv(core_pad, core_path)
    to_csv(neigh, neigh_path)
    to_csv(extra_neighs, e_neigh_path)


In [22]:
meshes = []
for c in range(len(corner_loc_lattices)):
    corner_mesh_path = core_path = os.path.relpath('../data/bmc_templates/bmc_subtiles/t_' + f'{c:02}' + '.obj')
    corner_mesh = tm.load(corner_mesh_path)
    meshes.append(corner_mesh)

In [23]:
type(meshes[10])

trimesh.base.Trimesh

In [24]:
tile_corner_inds = []
for result in results:
    # find each corner in the list of unique corner arrangements
    corner_ind = np.array([np.where((r == uniq_corner_arang).all(1)) for r in result]).flatten()
    tile_corner_inds.append(corner_ind)

tile_corner_inds = np.array(tile_corner_inds)
print(tile_corner_inds)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  3  2  9]
 [ 0  0  3 ...  0  9  1]
 ...
 [17 16 14 ... 16  8 15]
 [17 16 17 ... 13 15  7]
 [17 16 17 ... 16 17 16]]


In [55]:
keys = ["OO", "XP", "XN", "YP", "YN", "ZP", "ZN"]
complete_profiles = []
for l in l_bis:
    # apply the stencils in all corners of all possible configurations
    p_s = [l.apply_stencil(corner_state[k]).flatten() for k in keys]

    # concatenate results into columns
    profile = np.c_[tuple(p_s)]

    complete_profiles.append(profile)

complete_profiles = np.array(complete_profiles)
print(complete_profiles.shape)

unique_profiles = []
for l, loc in zip(corner_neigh_lattices,corner_loc_lattices):
    # apply the stencils in all corners of all possible configurations
    p_s = [l.apply_stencil(corner_state[k]).flatten() for k in keys]

    # concatenate results into columns
    profile = np.c_[tuple(p_s)]

    # extract the profile of the origin of the corner only
    origin_profile = profile[loc.flatten()==1].flatten()

    unique_profiles.append(origin_profile)

unique_profiles = np.array(unique_profiles)
unique_profiles.shape

(256, 8, 7)


(18, 7)

In [26]:
corner_pos = np.array(np.where(np.ones((2,2,2)) == 1)).T - 0.5
print(corner_pos)

tiles_meshes = []
### loading meshes
for tile in tile_corner_inds:
    last_v_count = 0
    vertice_list = []
    face_list = []
    for c_ind, pos in zip(tile, corner_pos):
        # extract current mesh
        corner_mesh = meshes[c_ind]
        if type(corner_mesh) == tm.base.Trimesh:
            # append the vertices
            vertice_list.append(corner_mesh.vertices + pos)
            face_list.append(corner_mesh.faces + last_v_count)
            last_v_count += len(corner_mesh.vertices)

    vs = []
    fs = []
    if len(vertice_list):
        vs = np.vstack(vertice_list)
        fs = np.vstack(face_list)

    tile_mesh = tm.Trimesh(vs, fs)
    tiles_meshes.append(tile_mesh)


[[-0.5 -0.5 -0.5]
 [-0.5 -0.5  0.5]
 [-0.5  0.5 -0.5]
 [-0.5  0.5  0.5]
 [ 0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0.5]]


In [29]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

p = pv.Plotter(notebook=True)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    mesh = tiles_meshes[i]
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.2, show_scalar_bar=False, color="white")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#abd8ff', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, len(tiles_meshes)], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(4.363703305156274, 4.363703305156274, 4.363703305156274),
 (0.5, 0.5, 0.5),
 (0.0, 0.0, 1.0)]

In [10]:
lattice = corner_neigh_lattices[4]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = lattice.minbound - lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = lattice.unit 

# Add the data values to the cell data
grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([.9, 1.1], scalars="filled")
# extract mesh
mesh = threshed.extract_surface()
points = mesh.points
quad_faces = mesh.faces.reshape((-1, 5))[:, 1:]
tri_faces = np.c_[quad_faces[:,0:3], quad_faces[:,2:4], quad_faces[:,0]].reshape((-1, 3))
tri_faces_pv = np.c_[np.full(len(tri_faces), 3), tri_faces]
# pv.PolyData(points, tri_faces)
tri_mesh = tm.Trimesh(points, tri_faces)
# out = 
# out

with open(obj_path, 'w') as file:
    file.write(tm.exchange.obj.export_obj(tri_mesh))
# tri_mesh.show()
# tri_faces_pv
# tri_faces

NameError: name 'obj_path' is not defined

### Credits

In [ ]:
__aut__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"